In [1]:
import databaseOps, fields
import numpy as np
import json

In [2]:
conn = databaseOps.connectToDb()
cur = conn.cursor()

In [3]:
def score(aRows, bRows):
    numMatches, editDist, naiveDist = compare(aRows, bRows)
    score = numMatches - editDist - naiveDist
    return score/len(aRows)

# rows need to be ordered by row num
def compare(aRow, bRow):
    numMatches = 0
    editDists, naiveDists = [], []
    numFields = min(len(aRow), len(bRow))
    if numFields == 0:
        return 0
    for j in range(numFields):
        numMatches += sum([1 for i in range(numFields) if aRow[i] == bRow[i]])
        editDists.append(sum([editDistance(str(aRow[i]), str(bRow[i])) for i in range(numFields)]))
        naiveDists.append(sum([naiveEdit(str(aRow[i]), str(bRow[i])) for i in range(numFields)]))
#     print(numMatches, sum(editDists), sum(naiveDists))
    return (numMatches - sum(editDists) - sum(naiveDists))/numFields #more negative = more unlike
    return numMatches - sum(editDists) - sum(naiveDists) #more negative = more unlike

# aRow & bRow are dictionaries
def compareByCols(aRow, bRow):
    numMatches = 0
    editDists, naiveDists = [], []
    cols = list(aRow.keys())
    numCols = len(cols)
    for j in range(numCols):
        numMatches += sum([1 for i in range(numCols) if aRow[cols[i]] == bRow[cols[i]]])
        editDists.append(sum([editDistance(str(aRow[cols[i]]), str(bRow[cols[i]])) for i in range(numCols)]))
        naiveDists.append(sum([naiveEdit(str(aRow[cols[i]]), str(bRow[cols[i]])) for i in range(numCols)]))
#     print((numMatches - sum(editDists) - sum(naiveDists))/numCols)
    return (numMatches - sum(editDists) - sum(naiveDists))/numCols #more negative = more unlike
#     return numMatches - sum(editDists) - sum(naiveDists) #more negative = more unlike

# Shortest edit distance (Levenstein)
def editDistance(aField, bField):
    if aField == bField:
        return 0
    n, m = len(aField), len(bField)
    if aField == "None":
        return m/m
    if bField == "None":
        return n/n
    dp = [[0 for x in range(n+1)] for x in range(m+1)] 
    for i in range(m+1): 
        for j in range(n+1):
            if i == 0: 
            	dp[i][j] = j
            elif j == 0: 
                dp[i][j] = i
            elif aField[j-1] == bField[i-1]: 
                dp[i][j] = dp[i-1][j-1] 
            else: 
                dp[i][j] = 1 + min(dp[i][j-1], dp[i-1][j], dp[i-1][j-1])    
  
    return dp[m][n]/(n+m) #normalized is better

# delete from end of bField until match beginning of aField, then add rest of aField
def naiveEdit(aField, bField):
    if aField == bField:
        return 0
    n, m = len(aField), len(bField)
    if aField == "None":
        return m/m
    if bField == "None":
        return n/n
    if (len(aField) > len(bField)):
        a, b = aField, bField
    else:
        b, a = aField, bField
    numEdits = 0
    while aField.find(bField) != 0 and len(bField) > 0:
        bField = bField[:-1]
        numEdits += 1
    numEdits += len(aField) - len(bField)
    return numEdits/(n+m)

In [41]:
### compare just head; using rownumber and dictionary fields

fourA = [
'887546/000095011717000076/0000950117-17-000076', 
'893739/000121102217000006/0001211022-17-000006', 
'1305168/000089924317006824/0000899243-17-006824', 
'1653649/000120919117010174/0001209191-17-010174', 
'893739/000129672017000004/0001296720-17-000004', 
'1173479/000117347917000003/0001173479-17-000003']

# fourA = ['1173479/000117347917000003/0001173479-17-000003']

aTo4 = dict()
for idx in range(len(fourA)):
    aHead = getRows(fourA[idx], "form4Head")
    query = "select B.accNum as bAcc from(select * from form4head group by accNum) A, (select * from form4head group by accNum) B where A.accNum='"+ fourA[idx] + "' and B.documentType = '4' and A.dateOfOriginalSubmission = B.filedDate;" 

    matches = cur.execute(query).fetchall()
    probDist = dict()
    headScores, totalScores = dict(), dict()
    for match in matches:
        bAcc = match[0]
        bHead = getRows(bAcc, "form4Head")
        
        mHeadScores = np.zeros(len(bHead))
        
        for aRow in aHead:
            for bIdx in range(len(bHead)):
                bRow = bHead[bIdx]
                if aRow['rowNumber'] == bRow['rowNumber']:
                    mHeadScores[bIdx] = compareByCols(aRow, bRow)    

        totalScore = sum(mHeadScores)
        headScores[bAcc], totalScores[bAcc] = mHeadScores, totalScore
    
    scores = totalScores.values()
    minScore = min(scores)
    total = 0
    for key in totalScores.keys():
        if minScore < 0:
            totalScores[key] += abs(minScore) + 0.0000000000001 #prevent from nan
        total += totalScores[key]
    for key in totalScores.keys():
        probDist[key] = totalScores[key]/total 
        
    values = sorted(probDist.values(), reverse=True)
    aTo4[fourA[idx]] = dict()
    for key in probDist.keys():
        if probDist[key] in values[:4]:
            aTo4[fourA[idx]][key] = probDist[key]

In [26]:
print(json.dumps(aTo4, indent=2))

{
  "1653649/000120919117010174/0001209191-17-010174": {
    "1423824/0001209191-17-010156": 0.012833208304662187,
    "880266/0001209191-17-010011": 0.014425832546611226,
    "1653649/0001209191-17-010146": 0.0494546320307282,
    "1391094/0001225208-17-002902": 0.010815517748141927
  },
  "887546/000095011717000076/0000950117-17-000076": {
    "887546/0000950117-17-000073": 0.01669836001412758,
    "1365555/0001365555-17-000013": 0.0065316053061594955,
    "887546/0000950117-17-000072": 0.020426583195587676,
    "1620083/0001174947-17-000112": 0.007327724040058352
  },
  "1173479/000117347917000003/0001173479-17-000003": {
    "1267238/0001209191-17-020499": 0.02119676518588861,
    "1173479/0001173479-17-000002": 0.06643694052404525,
    "1267238/0001209191-17-020492": 0.023159400378675827,
    "1208099/0000014272-17-000084": 0.029510378823633065
  },
  "1305168/000089924317006824/0000899243-17-006824": {
    "1305168/0000899243-17-004732": 0.018467035581399113,
    "1305168/0000899

In [4]:
from fields import getHeadFields, getDTFields, getNDTFields, getFootFields, filterFields

# return all rows with accNum in table in dictionary form (keys = table field)
def getRows(accNum, table, transactionType=None):
    if transactionType is not None:
        cur.execute("select * from "+ table +" where accNum = '" + accNum + "' and type = '" + transactionType + "';")
    else:
        cur.execute("select * from "+ table +" where accNum = '" + accNum + "';")
    rows = [dict(zip([col[0] for col in cur.description], row)) for row in cur.fetchall()]
    return rows

headFields = getHeadFields()
headDropField = ['filedDate', 'dateOfOriginalSubmission', 'accNum', 'headId', 'documentType', 'rptOwnerFormType', 'changedDate']
headFields = filterFields(headFields, headDropField)

ndtFields = getNDTFields()
ndtDropField = ['accNum', 'nDTId', 'footNoteId', 'rowNumber', 'type']
ndtFields = filterFields(ndtFields, ndtDropField)
dtFields = getDTFields()

dtDropField = ['accNum', 'dTId', 'footNoteId', 'rowNumber', 'type']
dtFields = filterFields(dtFields, dtDropField)

footFields = getFootFields()
footDropField = ['accNum', 'fId', 'footNoteId', 'rowNumber', 'type']
footFields = filterFields(footFields, footDropField)

In [83]:
def getChangedFields(aRow, bRow, tblFields):
    changedFields = dict()
    for field in tblFields:
        if bRow is None or bRow == [] or bRow[field] == 'null' or bRow[field] is None:
            if aRow[field] == "null" or aRow[field] is None:
                continue
            else:
                changedFields[field] = (aRow[field], '') #(after, before)
        elif (aRow[field] == "null" or aRow[field] is None) and (bRow[field] != 'null' or bRow[field] is not None):
            changedFields[field] = ('', bRow[field]) #(after, before)
        elif aRow[field] != bRow[field]:
            changedFields[field] = (aRow[field], bRow[field]) #(after, before)
#     print(changedFields)
    return changedFields

In [70]:
# given aRows, return prob dist of bRows being match for each aRow
# dict[aRow][probs] = prob
def getMatchProbDist(aRows, bRows, identifier): #4/a, 4
    probDist = dict()
    for aRow in aRows:
        probDist[aRow[identifier]] = dict()

    # if only 1 transaction in each
    if len(aRows) == 1 and len(bRows) == 1:
        probDist[aRows[0][identifier]][bRows[0][identifier]] = 1
        return probDist
    
    # score matching each 4a with each 4
    for aRow in aRows:
        transacMatchScores = dict()
        for bRow in bRows:
            transacMatchScores[bRow[identifier]] = compareByCols(aRow, bRow)
            
        #normalize scores 
        total = 0
        scores = transacMatchScores.values()
        minScore = min(scores)
        for key in transacMatchScores.keys():
            if minScore < 0:
                transacMatchScores[key] += abs(minScore) + 0.0000000001 #prevent from nan
            total += transacMatchScores[key]
        #
        for key in transacMatchScores.keys():
            probDist[aRow[identifier]][key] = transacMatchScores[key]/total
        
    #print('probDist', json.dumps(probDist, indent=2))
    return probDist

In [36]:
import pulp
    
def getOptMatches(probDist):
    fourAIDs = list(probDist.keys())
    fourIDs = list(probDist[fourAIDs[0]].keys())
    n, m = len(fourAIDs), len(fourIDs)
    allVars, variables = [], []
    for i in range(n):
        x = [str(i) + '|' + str(j) for j in range(m)]
        variables.append(x)

    lp = pulp.LpProblem("max weighted matching", pulp.LpMaximize)
    lstVars = []
    for j in range(len(variables)):
        a = [pulp.LpVariable(variables[j][i], lowBound = 0, upBound = 1, cat='Integer') for i in range(len(variables[j]))]
        lstVars.append(a)
        allVars += a
        
    objFunc, varWeightMap = [], {}
    for var in allVars:
        varName = var.name.split('|')
        fourAID, fourID = fourAIDs[int(varName[0])], fourIDs[int(varName[1])]
        objFunc += [(var, probDist[fourAID][fourID])] 
        varWeightMap[var.name] = probDist[fourAID][fourID]
    objFunc = pulp.LpAffineExpression(objFunc)
    lp += objFunc

    for j in range(len(lstVars)):
        cons = [(var, 1) for var in lstVars[j]]
        lp += pulp.LpAffineExpression(cons) <= 1 #constraints
    lp.solve()
    # for variable in lp.variables():
    #     print("{} = {}".format(variable.name, variable.varValue))
    
    chosenVars = [(var.name, varWeightMap[var.name]) for var in lp.variables() if var.varValue == 1]
    chosenVars = sorted(chosenVars, key=lambda tup: tup[1], reverse=True) #sort by prob of matching
    
    allMatches, unmatched = [], []
    for i in range(n): #only top m matches
        chosenName = chosenVars[i][0].split('|')
        if i < m:
            allMatches.append((fourAIDs[int(chosenName[0])], fourIDs[int(chosenName[1])]))
        else:
            unmatched.append(fourAIDs[int(chosenName[0])])
#     print('allMatches', allMatches)
#     print('unmatched', unmatched)
    return allMatches, unmatched
    

In [86]:
# what fields was changed? 

# get matches: rptOwner & date
# for each possible: diff in head, diff in dt, diff in ndt -> {fieldname: (before/4, after/4a)}

#fourAs = ['1173479/000117347917000003/0001173479-17-000003'] #4/a: 4 ndt, 5 ndh; 4: 4 ndh **mix match case

#fourAs = ['893739/000121102217000006/0001211022-17-000006', '893739/000129672017000004/0001296720-17-000004'] # 4/a has 1 more dt **additional entry case

fourAs = ['887546/000095011717000076/0000950117-17-000076', '1653649/000120919117010174/0001209191-17-010174', '1305168/000089924317006824/0000899243-17-006824', '1584738/000114420417020939/0001144204-17-020939', '1689923/000089924317009598/0000899243-17-009598', '1405073/000120919117020624/0001209191-17-020624', '1310215/000005947817000106/0000059478-17-000106'] #same # of transactions/holdings; value updates only

diff = dict()
for idx in range(len(fourAs)):
    fourA = fourAs[idx]

    aHead = getRows(fourA, "form4Head")
    
    aDTT = getRows(fourA, "form4dT", "transaction")
    aDTH = getRows(fourA, "form4dT", "holding")
    aNDTT = getRows(fourA, "form4ndT", "transaction")    
    aNDTH = getRows(fourA, "form4ndT", "holding")
    
    aTransacs = [aDTT, aDTH, aNDTT, aNDTH]
    labels = ["dt", "dt", "ndt", "ndt"]
    tblFields = [dtFields, dtFields, ndtFields, ndtFields]
    identifiers = ["dTId", "dTId", "nDTId", "nDTId"]
    
    aFoot = getRows(fourA, "form4footnote")    
    
    query = "select B.accNum as bAcc from form4head A, form4head B where A.accNum='"+ fourA + "' and B.documentType = '4' and A.dateOfOriginalSubmission = B.filedDate and A.rptOwnerName = B.rptOwnerName;" 

    matches = cur.execute(query).fetchall() #grouped by [(aAcc, bAcc)]
    matchDiff, lenChanges = dict(), 0
    for match in matches:
        four = match[0]
        
        bHead = getRows(four, "form4Head")

        bDTT = getRows(four, "form4dT", "transaction")
        bDTH = getRows(four, "form4dT", "holding")
        bNDTT = getRows(four, "form4ndT", "transaction")    
        bNDTH = getRows(four, "form4ndT", "holding")        
        
        bFoot = getRows(four, "form4footnote")
        bTransacs = [bDTT, bDTH, bNDTT, bNDTH]
        
        thisMatchDiff = dict() 
        #what field changed in head, dt, ndt, footnote??
        thisMatchDiff["head"], thisMatchDiff["dt"], thisMatchDiff["ndt"] = dict(), dict(), dict()
        for aRow in aHead:
            for bRow in bHead:
                if aRow['rowNumber'] == bRow['rowNumber']:
                    changedFields = getChangedFields(aRow, bRow, headFields)
                    if len(changedFields) != 0:
                        thisMatchDiff["head"][aRow['rowNumber']] = changedFields
                    break

        for idx in range(len(aTransacs)):
            aTransac, bTransac = aTransacs[idx], bTransacs[idx]
            label, fields, identifier = labels[idx], tblFields[idx], identifiers[idx]
            if len(aTransac) != 0:
                if len(aTransac) == len(bTransac):            
                    for aRow in aTransac:
                        for bRow in bTransac:
                            if aRow['rowNumber'] == bRow['rowNumber']:
                                changedFields = getChangedFields(aRow, bRow, fields)
                                if len(changedFields) != 0:
                                    thisMatchDiff[label][str(aRow['rowNumber']) + '-' + aRow['type']] = changedFields
                                break
                elif len(bTransac) == 0:
                    for aRow in aTransac:
                        thisMatchDiff[label][str(aRow['rowNumber']) + '-' + aRow['type']] = getChangedFields(aRow, None, fields)
                else:
                    probDist = getMatchProbDist(aTransac, bTransac, identifier) #4/a, 4
                    optMatches, unmatched = getOptMatches(probDist) #get matches

                    matchA = [match[0] for match in optMatches]
                    matchB = [match[1] for match in optMatches]
                    #get rows corresponding to matchs 
                    aRows = [aRow for aRow in aTransac if aRow[identifier] in matchA]
                    bRows = [bRow for bRow in bTransac if bRow[identifier] in matchB]
                    unMatchedRows = [aRow for aRow in aTransac if aRow[identifier] in unmatched]
                    
                    for matchIdx in range(len(optMatches)):
                        aRow, bRow = aRows[matchIdx], bRows[matchIdx]
                        changedFields = getChangedFields(aRow, bRow, fields)
                        if len(changedFields) != 0:
                            thisMatchDiff[label][str(aRow['rowNumber']) + '-' + aRow['type']] = changedFields
                    for aRow in unMatchedRows:
                        thisMatchDiff[label][str(aRow['rowNumber']) + '-' + aRow['type']] = getChangedFields(aRow, None, fields)
        #matchDiff[four] = thisMatchDiff #display changes with all possible 4's
        
        #only display best matched form 4
        thisLenChanges = len(thisMatchDiff["head"]) + len(thisMatchDiff["dt"]) + len(thisMatchDiff["ndt"])
        if matchDiff == {} or (matchDiff != {} and thisLenChanges < lenChanges):
            thisMatchDiff["accNum"] = four
            matchDiff = thisMatchDiff
            lenChanges = thisLenChanges  
            
    diff[fourA] = matchDiff
print(json.dumps(diff, indent=2))

{
  "1653649/000120919117010174/0001209191-17-010174": {
    "dt": {},
    "head": {
      "1": {
        "rptOwnerFilmNum": [
          "17601353",
          "17601231"
        ]
      }
    },
    "accNum": "1653649/0001209191-17-010146",
    "ndt": {
      "1-transaction": {
        "securityTitle": [
          "<value>Common Stock</value><footnoteId id=\"F1\"/><footnoteId id=\"F2\"/>",
          "<value>Common Stock</value><footnoteId id=\"F1\"/>"
        ]
      }
    }
  },
  "887546/000095011717000076/0000950117-17-000076": {
    "dt": {},
    "head": {
      "1": {
        "rptOwnerFilmNum": [
          "17570785",
          "17567485"
        ]
      }
    },
    "accNum": "887546/0000950117-17-000072",
    "ndt": {
      "1-transaction": {
        "sharesOwnedFollowingTransaction": [
          "<value>5269</value><footnoteId id=\"F1\"/>",
          "<value>4785</value>"
        ]
      }
    }
  },
  "1689923/000089924317009598/0000899243-17-009598": {
    "dt": {},
    "head

In [3]:
import sys
!{sys.executable} -m pip install pandas

    100% |████████████████████████████████| 14.4MB 788kB/s  a 0:00:011
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
### compare head, dt, and ndt

fourA = [
'887546/000095011717000076/0000950117-17-000076', 
'893739/000121102217000006/0001211022-17-000006', 
'1305168/000089924317006824/0000899243-17-006824', 
'1653649/000120919117010174/0001209191-17-010174', 
'893739/000129672017000004/0001296720-17-000004', 
'1173479/000117347917000003/0001173479-17-000003'] #[list of accNum]
#fourA = ['1305168/000089924317006824/0000899243-17-006824'] #[list of accNum]
#fourA = ['1653649/000120919117010174/0001209191-17-010174'] #need to debug

aTo4 = dict()
for idx in range(len(fourA)):
    aHead = cur.execute("select * from form4Head where accNum = '" + fourA[idx]+ "';").fetchall()
    aDT = cur.execute("select * from form4dT where accNum = '" + fourA[idx]+ "';").fetchall()
    aNDT = cur.execute("select * from form4ndT where accNum = '" + fourA[idx]+ "';").fetchall()

    query = "select B.accNum as bAcc from(select * from form4head group by accNum) A, (select * from form4head group by accNum) B where A.accNum='"+ fourA[idx] + "' and B.documentType = '4' and A.dateOfOriginalSubmission = B.filedDate;" 

    matches = cur.execute(query).fetchall() #grouped by [(aAcc, bAcc)]
    probDist = dict()
    headScores, dtScores, ndtScores, totalScores = dict(), dict(), dict(), dict()
    for match in matches:
        bAcc = match[0]
        bHead = cur.execute("select * from form4Head where accNum = '" + bAcc + "';").fetchall()
        bDT = cur.execute("select * from form4dT where accNum = '" + bAcc + "';").fetchall()
        bNDT = cur.execute("select * from form4ndT where accNum = '" + bAcc + "';").fetchall()
        
        if (len(aDT) != 0 and len(bDT) == 0) or (len(aNDT) != 0 and len(bNDT) == 0):
            continue
        
        scoreLen = max(len(bHead), len(bDT), len(bNDT))
        mHeadScores, mDtScores, mNdtScores = np.zeros(scoreLen), np.zeros(scoreLen), np.zeros(scoreLen)
        
        for a in aHead:
            for j in range(len(bHead)):
                b = bHead[j]
                mHeadScores[j] = compare(a, b)
        
#         for a in aDT:
#             for j in range(len(bDT)): #if dt is empty then score is 0 (maybe ndt update?)
#                 b = bDT[j]
#                 mDtScores[j] = compare(a, b)
#         for a in aNDT:
#             for j in range(len(bNDT)):
#                 b = bNDT[j]
#                 mNdtScores[j] = compare(a, b)

        totalScore = sum([mHeadScores[p] + mDtScores[p] + mNdtScores[p] for p in range(len(mHeadScores))])
        headScores[bAcc], dtScores[bAcc], ndtScores[bAcc], totalScores[bAcc] = mHeadScores, mDtScores, mNdtScores, totalScore
    scores = totalScores.values()
    minScore = min(scores)
    total = 0
    for key in totalScores.keys():
        if minScore < 0:
            totalScores[key] += abs(minScore) + 1 #prevent from nan
        total += totalScores[key]
    for key in totalScores.keys():
        probDist[key] = totalScores[key]/total
        
    values = sorted(probDist.values(), reverse=True)
    aTo4[fourA[idx]] = dict()
    for key in probDist.keys():
        if probDist[key] in values[:4]:
            aTo4[fourA[idx]][key] = probDist[key]